In [28]:
import torch
from utils.config import Config
import torch.nn as nn
import torch.nn.functional as F
config = Config()

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=kernel_size // 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)  # (B, 1, H, W)
        max_out, _ = torch.max(x, dim=1, keepdim=True)  # (B, 1, H, W)
        attn = torch.cat([avg_out, max_out], dim=1)  # (B, 2, H, W)
        attn = self.conv(attn)  # (B, 1, H, W)
        attn = self.sigmoid(attn)  # (B, 1, H, W)
        return x * attn  # Apply attention

class InkDetector(nn.Module):
    def __init__(self, config: Config):
        super(InkDetector, self).__init__()
        self.features = nn.Sequential(
            # Input: (B, 1, 8, 32, 32)
            nn.Conv3d(1, 32, kernel_size=(3, 4, 4), padding=1, bias=False),  # (B, 32, 6, 31, 31)
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True),
            nn.Dropout3d(config.model.conv1_drop),
            
            nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=1, bias=False),  # (B, 64, 4, 31, 31)
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(1, 2, 2)),  # (B, 64, 6, 15, 15)
            
            nn.Conv3d(64, 96, kernel_size=(4, 3, 3), padding=1, bias=False),  # (B, 96, 1, 15, 15)
            nn.BatchNorm3d(96),
            nn.ReLU(inplace=True),
            nn.Dropout3d(config.model.conv2_drop),
            nn.MaxPool3d(kernel_size=(1, 2, 2)),  # (B, 96, 5, 7, 7)
            
            nn.Conv3d(96, 128, kernel_size=(1, 3, 3), padding=1, bias=False),  # (B, 128, 1, 7, 7)
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.Dropout3d(config.model.conv1_drop),
            nn.MaxPool3d(kernel_size=(1, 2, 2)),  # (B, 128, 1, 3, 3)
            nn.Dropout3d(config.model.conv2_drop),
            
            nn.AdaptiveAvgPool3d(1)  # (B, 128, 1, 1, 1)
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.Dropout(config.model.fc1_drop),
            nn.Linear(64, 32, bias=False),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),
            nn.Dropout(config.model.fc2_drop),
            nn.Linear(32, 1)  # Keep bias for final output layer
        )

    def _register_hooks(self):
        def hook(module, input, output):
            self.activations[module] = output.detach()

        for layer in self.features:
            if not isinstance(layer, (nn.Dropout3d, nn.BatchNorm3d)):
                layer.register_forward_hook(hook)
        for layer in self.classifier:
            if not isinstance(layer, (nn.Dropout, nn.BatchNorm1d)):
                layer.register_forward_hook(hook)


model = InkDetector(config).to('cuda')  # Instantiate your model (replace InkDetector with your class)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_params}")


Total trainable parameters: 399713


In [36]:
class CBAM3D(nn.Module):
    def __init__(self, channels, reduction=16, kernel_size=3):
        super(CBAM3D, self).__init__()

        # Channel Attention
        self.avg_pool = nn.AdaptiveAvgPool3d(1)
        self.max_pool = nn.AdaptiveMaxPool3d(1)

        self.shared_mlp = nn.Sequential(
            nn.Conv3d(channels, channels // reduction, kernel_size=1, bias=False),
            nn.ReLU(),
            nn.Conv3d(channels // reduction, channels, kernel_size=1, bias=False)
        )
        self.sigmoid_channel = nn.Sigmoid()

        # Spatial Attention
        self.conv_spatial = nn.Conv3d(2, 1, kernel_size=kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid_spatial = nn.Sigmoid()

    def forward(self, x):
        # --- Channel Attention ---
        avg_out = self.shared_mlp(self.avg_pool(x))
        max_out = self.shared_mlp(self.max_pool(x))
        channel_attn = self.sigmoid_channel(avg_out + max_out)
        x = x * channel_attn

        # --- Spatial Attention ---
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        spatial_attn = self.sigmoid_spatial(self.conv_spatial(torch.cat([avg_out, max_out], dim=1)))
        x = x * spatial_attn

        return x

class InkDetector(nn.Module):
    def __init__(self, config: Config):
        super(InkDetector, self).__init__()

        self.features = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=(3, 4, 4), padding=1, bias=False),  # (B, 32, 8, 31, 31)
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True),
            CBAM3D(32),
            nn.Dropout3d(config.model.conv1_drop),

            nn.Conv3d(32, 96, kernel_size=(3, 3, 3), padding=1, bias=False),  # (B, 96, 8, 31, 31)
            nn.BatchNorm3d(96),
            nn.ReLU(inplace=True),
            CBAM3D(96),
            nn.MaxPool3d(kernel_size=(2, 2, 2)),  # (B, 96, 4, 15, 15)

            nn.Conv3d(96, 128, kernel_size=(3, 3, 3), padding=1, bias=False),  # (B, 128, 4, 15, 15)
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            CBAM3D(128),
            nn.MaxPool3d(kernel_size=(2, 2, 2)),  # (B, 128, 2, 7, 7)

            nn.AdaptiveAvgPool3d(1)  # (B, 128, 1, 1, 1)
        )


        self.classifier = nn.Sequential(
            nn.Flatten(),  # (B, 128)
            nn.Linear(128, 64, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.Dropout(config.model.fc1_drop),

            nn.Linear(64, 32, bias=False),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),
            nn.Dropout(config.model.fc2_drop),

            nn.Linear(32, 1)  # Output: (B, 1)
        )
        self.activations = {}
        self._register_hooks()

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


    def _register_hooks(self):
        def hook(module, input, output):
            self.activations[module] = output.detach()

        for layer in self.features:
            if not isinstance(layer, (nn.Dropout3d, nn.BatchNorm3d)):
                layer.register_forward_hook(hook)
        for layer in self.classifier:
            if not isinstance(layer, (nn.Dropout, nn.BatchNorm1d)):
                layer.register_forward_hook(hook)

model = InkDetector(config).to('cuda')  # Instantiate your model (replace InkDetector with your class)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_params}")


Total trainable parameters: 430723


In [23]:
import vesuvius
from vesuvius import Volume
import matplotlib.pyplot as plt

scroll = Volume(type="Scroll", scroll_id=3, energy=53, resolution=7.91)
print(scroll.shape(0))

(11174, 3340, 3440)


In [ ]:
import vesuvius
from vesuvius import Volume
import matplotlib.pyplot as plt



An error occurred while initializing the Volume class: URL not found for scroll: 4, energy: 53, resolution: 7.91, segment: 20231210132040
Load the canonical scroll 1 with Volume(type="scroll", scroll_id=1, energy=54, resolution=7.91)
If loading another part of the same physical scroll use for instance Volume(type="scroll", scroll_id="1b", energy=54, resolution=7.91)
Load a segment (e.g. 20230827161847) with Volume(type="segment", scroll_id=1, energy=54, resolution=7.91, segment_id=20230827161847)


ValueError: URL not found for scroll: 4, energy: 53, resolution: 7.91, segment: 20231210132040